In [1]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
import pandas as pd
from nltk.stem import PorterStemmer
import nltk
import contractions
import re

stop_words = nltk.corpus.stopwords.words('english')

In [2]:
import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 200

corpus = ['The sky is blue and beautiful.',
          'Love this blue and beautiful sky!',
          'The quick brown fox jumps over the lazy dog.',
          "A king's breakfast has sausages, ham, bacon, eggs, toast and beans",
          'I love green eggs, ham, sausages and bacon!',
          'The brown fox is quick and the blue dog is lazy!',
          'The sky is very blue and the sky is very beautiful today',
          'The dog is lazy but the brown fox is quick!'    
]
labels = ['weather', 'weather', 'animals', 'food', 'food', 'animals', 'weather', 'animals']

corpus = np.array(corpus)
corpus_df = pd.DataFrame({'Document': corpus, 
                          'Category': labels})
corpus_df = corpus_df[['Document', 'Category']]
corpus_df

,Document,Category
0,The sky is blue and beautiful.,weather
1,Love this blue and beautiful sky!,weather
2,The quick brown fox jumps over the lazy dog.,animals
3,"A king's breakfast has sausages, ham, bacon, eggs, toast and beans",food
4,"I love green eggs, ham, sausages and bacon!",food
5,The brown fox is quick and the blue dog is lazy!,animals
6,The sky is very blue and the sky is very beautiful today,weather
7,The dog is lazy but the brown fox is quick!,animals


In [3]:
def convert_lower(text):
    text = text.lower()
    return text 

def expand_contractions(text):
    text = contractions.fix(text)
    return text 

def remove_special_character(text):
    text = text.replace('...',' ')
    text = re.sub('[^a-zA-Z\s]','',text)
    return text

def stemming(text):
    text = [ps.stem(w) for w in text.split(' ')]
    return ' '.join(text)

def stopword_removal(text):
    text = [w for w in text.split(' ') if w not in stop_words]
    return ' '.join(text)

def special_mention(text):
    text = re.sub('@\w+','',text)
    return text 

In [4]:
def clean_text(text):
    text = convert_lower(text)

    text = expand_contractions(text)

   # text = special_mention(text)

    text = remove_special_character(text)

    text = stemming(text)

    text = stopword_removal(text)
    
    text = text.strip()

    return text

In [5]:
corpus_df['clean_text'] = corpus_df['Document'].apply(lambda x: clean_text(x))

In [6]:
corpus_df

,Document,Category,clean_text
0,The sky is blue and beautiful.,weather,sky blue beauti
1,Love this blue and beautiful sky!,weather,love thi blue beauti sky
2,The quick brown fox jumps over the lazy dog.,animals,quick brown fox jump lazi dog
3,"A king's breakfast has sausages, ham, bacon, eggs, toast and beans",food,king breakfast ha sausag ham bacon egg toast bean
4,"I love green eggs, ham, sausages and bacon!",food,love green egg ham sausag bacon
5,The brown fox is quick and the blue dog is lazy!,animals,brown fox quick blue dog lazi
6,The sky is very blue and the sky is very beautiful today,weather,sky veri blue sky veri beauti today
7,The dog is lazy but the brown fox is quick!,animals,dog lazi brown fox quick


In [7]:
doc = list(corpus_df['clean_text'].values)
doc

['sky blue beauti',
 'love thi blue beauti sky',
 'quick brown fox jump lazi dog',
 'king breakfast ha sausag ham bacon egg toast bean',
 'love green egg ham sausag bacon',
 'brown fox quick blue dog lazi',
 'sky veri blue sky veri beauti today',
 'dog lazi brown fox quick']

In [27]:
corpus = ''
for s in doc:
    corpus = corpus + s

In [28]:
corpus

'sky blue beautilove thi blue beauti skyquick brown fox jump lazi dogking breakfast ha sausag ham bacon egg toast beanlove green egg ham sausag baconbrown fox quick blue dog lazisky veri blue sky veri beauti todaydog lazi brown fox quick'

In [30]:
import gensim

In [33]:
print(gensim.__version__)

4.3.0


In [31]:
print(dir(gensim))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_matutils', 'corpora', 'interfaces', 'logger', 'logging', 'matutils', 'models', 'parsing', 'similarities', 'topic_coherence', 'utils']


In [29]:
from gensim.summarization.summarizer import summarize

ModuleNotFoundError: No module named 'gensim.summarization'

In [ ]:
summarize(corpus)

## Count Vectorizer

In [8]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [21]:
cnt = CountVectorizer(ngram_range=(1,2),min_df=2)

In [22]:
data = cnt.fit_transform(doc)

In [23]:
data

<8x17 sparse matrix of type '<class 'numpy.int64'>'
	with 44 stored elements in Compressed Sparse Row format>

In [24]:
count_df = pd.DataFrame(data.toarray(),columns=sorted(cnt.vocabulary_.keys()))

In [25]:
count_df.shape

(8, 17)

In [18]:
count_df.shape

(8, 56)

In [40]:
count_df.shape

(8, 23)

In [19]:
doc

['sky blue beauti',
 'love thi blue beauti sky',
 'quick brown fox jump lazi dog',
 'king breakfast ha sausag ham bacon egg toast bean',
 'love green egg ham sausag bacon',
 'brown fox quick blue dog lazi',
 'sky veri blue sky veri beauti today',
 'dog lazi brown fox quick']

In [26]:
count_df

,bacon,beauti,blue,blue beauti,brown,brown fox,dog,dog lazi,egg,fox,fox quick,ham,lazi,love,quick,sausag,sky
0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1
2,0,0,0,0,1,1,1,0,0,1,0,0,1,0,1,0,0
3,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0
4,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0
5,0,0,1,0,1,1,1,1,0,1,1,0,1,0,1,0,0
6,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2
7,0,0,0,0,1,1,1,1,0,1,1,0,1,0,1,0,0


In [43]:
cnt.vocabulary_.keys()

dict_keys(['sky', 'blue', 'beauti', 'love', 'thi', 'quick', 'brown', 'fox', 'jump', 'lazi', 'dog', 'king', 'breakfast', 'ha', 'sausag', 'ham', 'bacon', 'egg', 'toast', 'bean', 'green', 'veri', 'today'])

## Tf Idf Vectorizer : TF (Term Frequency ) IDF (Inverse Document Frequency)

In [44]:
tfidf = TfidfVectorizer()

In [45]:
tf_data = tfidf.fit_transform(doc)

In [46]:
tf_df = pd.DataFrame(tf_data.toarray(), columns=sorted(tfidf.vocabulary_.keys()))

In [47]:
tf_df.head()

,bacon,bean,beauti,blue,breakfast,brown,dog,egg,fox,green,...,king,lazi,love,quick,sausag,sky,thi,toast,today,veri
0,0.000000,0.00000,0.600978,0.526925,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.600978,0.000000,0.00000,0.0,0.0
1,0.000000,0.00000,0.407444,0.357238,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.472170,0.000000,0.000000,0.407444,0.563396,0.00000,0.0,0.0
2,0.000000,0.00000,0.000000,0.000000,0.00000,0.380362,0.380362,0.000000,0.380362,0.000000,...,0.00000,0.380362,0.000000,0.380362,0.000000,0.000000,0.000000,0.00000,0.0,0.0
3,0.299898,0.35784,0.000000,0.000000,0.35784,0.000000,0.000000,0.299898,0.000000,0.000000,...,0.35784,0.000000,0.000000,0.000000,0.299898,0.000000,0.000000,0.35784,0.0,0.0
4,0.394554,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.394554,0.000000,0.470784,...,0.00000,0.000000,0.394554,0.000000,0.394554,0.000000,0.000000,0.00000,0.0,0.0


In [48]:
doc

['sky blue beauti',
 'love thi blue beauti sky',
 'quick brown fox jump lazi dog',
 'king breakfast ha sausag ham bacon egg toast bean',
 'love green egg ham sausag bacon',
 'brown fox quick blue dog lazi',
 'sky veri blue sky veri beauti today',
 'dog lazi brown fox quick']

In [49]:
import numpy as np

In [50]:
1/3

0.3333333333333333

In [55]:
np.log1p(8/3)

1.2992829841302609

In [56]:
(1/3)*np.log1p(8/3)

0.4330943280434203

In [71]:
np.log(8/8)

0.0

## Glove Embedding

In [57]:
import spacy

nlp =spacy.load('en_core_web_lg')

vectors= len(nlp.vocab.vectors)

In [60]:
vectors

514157

In [61]:
nlp('sky').vector.shape

(300,)

In [62]:
d={'the':[1,2,3],'sky':[2,3,4]}

In [63]:
d['sky']

[2, 3, 4]

In [64]:
nlp('the').vector

array([-5.1043e+00,  2.3496e+00,  3.2472e+00,  2.8424e+00,  1.1459e+01,
       -2.4137e+00,  5.1057e-01,  7.0312e+00,  3.6459e+00,  6.2332e-01,
        1.3633e+01,  4.5813e+00, -1.0584e+01,  1.2630e+00,  6.3362e-01,
        7.4645e+00,  6.1468e+00,  3.9474e-01,  1.4378e+00, -4.1540e+00,
        2.0000e+00, -3.8488e+00,  7.3414e-01,  2.1209e+00,  2.1068e+00,
        1.8713e+00, -7.8175e+00, -4.4352e+00,  2.2135e-01,  3.9262e+00,
        2.8473e+00,  2.0265e+00, -1.8189e+00, -9.2866e+00, -8.2191e+00,
       -1.7172e+00, -1.7196e+00,  3.9313e+00,  2.5888e+00,  8.2825e-01,
        1.3177e+00,  1.1566e+00, -5.2680e-01, -5.3276e-01, -1.2835e+00,
       -2.4743e-01, -3.7231e+00, -4.5196e-01, -3.3093e+00, -1.2523e+00,
       -3.0767e+00,  8.5522e+00, -1.6251e+00, -5.5941e+00,  5.3161e-01,
        5.5142e-01, -5.0327e-01,  1.8156e+00, -2.9011e+00, -1.8576e+00,
        1.4638e+00, -5.4034e+00, -8.1768e-01, -2.0847e+00,  1.6838e+00,
        1.6817e+00, -6.3836e+00, -3.2180e+00,  7.2152e+00,  3.52

In [65]:
(emb_sky + emb_beauti+ emb_blue) = 300+ 300+ 300 = 

300
300
300

ave

SyntaxError: cannot assign to expression (2681550646.py, line 1)

In [66]:
import numpy as np

In [67]:
np.array([1,2,3])+ np.array([1,1,1]) + np.array([100,100,100])

array([102, 103, 104])

In [80]:
def average_word_embedding(doc,feature_size):
    vector = np.zeros((feature_size),dtype='float64')
    total_words= len(doc)
    for word in doc:
        vector = np.add(vector,nlp(word).vector)
    vector = np.divide(vector,total_words)
    return vector

In [81]:
features = []
for sent in doc:
    features.append(average_word_embedding(sent.split(' '),))

In [85]:
features = np.array(features)

In [86]:
features.shape

(8, 300)

## FastText by FaceBook 

In [11]:
import gensim

from gensim.models.fasttext import FastText

ftm = FastText(doc, vector_size=20 )



ftm.wv['part'].shape

(20,)

In [12]:
# Get Embedding 
ftm.wv['part']

array([ 0.00306484,  0.00708508, -0.00173493, -0.00372327, -0.02049023,
        0.02521915, -0.01300305, -0.00493138, -0.00162844, -0.00261402,
        0.01097095, -0.00817131, -0.00747034, -0.00809746,  0.00032681,
       -0.00907891, -0.00239568,  0.01378425, -0.0096832 ,  0.00805305],
      dtype=float32)